In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
import altair as alt
import re

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score, precision_recall_curve, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from xgboost import XGBClassifier

#from sklearn.preprocessing import (Imputer, StandardScaler, MinMaxScaler) 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### [Link to data description](https://www.kaggle.com/c/titanic/data)

In [ ]:
import pandas as pd
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

In [ ]:
# Lets create copies of dataset to add new features
test_df = test.copy()
train_df = train.copy()

# Let's get a hang of what our dataset looks like? 

In [ ]:
train.info()

Age(177), Cabin(587) & Embarked(2) have missing values in the training data.

In [ ]:
test.info()

Age(86), Cabin(327), Fare(1) have missing values in the test data.

### Ok, So we have 9 features, 1 target, and PassengerId as the identifier. Not Bad

In [ ]:
train.head()

# Let's go through each of the features to understand them in detail.

## 1. Survived

There is no class imbalance in the classificaton task at hand

In [ ]:
train['Survived'].value_counts(normalize=True)

## 2. PClass
pclass: A proxy for socio-economic status (SES)
* 1st = Upper
* 2nd = Middle
* 3rd = Lower

In [ ]:
train['Pclass'].value_counts()

### Survival by Pclass (Socio-economic status)

In [ ]:
sns.catplot(x='Pclass', y='Survived', kind='bar', data=train)

### Well, If you're not rich and are going for a cruise, you might be the first one to die!

## 3. Name

<br>The patter in the name is "Last Name", "Title", "First Name Second Name" </br>
<br>Lets split this column on ",", "." and "(". </br>
<br>Title can also be use as a feature. </br>


In [ ]:
train_df['LastName'] = train['Name'].str.split(r",", expand=True, n=1).get(0)
test_df['LastName'] = test['Name'].str.split(r",", expand=True, n=1).get(0)
train_df['Title'] = train.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)
test_df['Title'] = test.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)

In [ ]:
sns.catplot(x='Title', y='Survived', data=train_df, kind='bar')
plt.xticks(rotation=90);

In [ ]:
def title_transform(x):
    if x == 'Mr':
        return x
    elif x in ['Mrs', 'Miss', 'Mme','Ms','Lady', 'Mlle', 'the Countess']:
        return 'Ms'
    elif x == 'Master':
        return x
    else:
        return 'Rare'

In [ ]:
train_df['Title'] = train_df.Title.apply(title_transform)
test_df['Title'] = test_df.Title.apply(title_transform)

In [ ]:
train_df['Title'].value_counts()

In [ ]:
sns.catplot(x='Title', y='Survived', data=train_df, kind='bar')

## 4. Sex

In [ ]:
train['Sex'].value_counts(normalize=True)

Women are more likely to survive shipwrecks. 

In [ ]:
sns.catplot(x='Sex', y='Survived', kind='bar', data=train)

In [ ]:
sns.catplot(x='Sex', y='Survived', kind='point', data=train)

## 5. SibSp
<br>sibsp: The dataset defines family relations in this way...</br>
<br>Sibling = brother, sister, stepbrother, stepsister</br>
<br>Spouse = husband, wife (mistresses and fiancés were ignored)</br>

In [ ]:
train['SibSp'].value_counts()

Data poinst for SibSp > 1 are very few, based on survival ratio for SibSp, possible features can be:
* 0, 1, {2,3,4,5,8..}
* 0, {1,2}, {3,4,5,8 ...}


In [ ]:
sns.catplot(x='SibSp', y='Survived', kind='bar', data=train)

In [ ]:
# First scheme
train_df['sibsp1'] = train_df['SibSp'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))
test_df['sibsp1'] = test_df['SibSp'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))

In [ ]:
# Second scheme
train_df['sibsp2'] = train_df['SibSp'].apply(lambda x: 0 if x == 0 else (1 if (x==1 | x==2) else 2))
test_df['sibsp2'] = test_df['SibSp'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))

In [ ]:
sns.catplot(x='sibsp2', y='Survived', kind='bar', data=train_df)

## 6. Parch
<br>parch: The dataset defines family relations in this way...</br>
<br>Parent = mother, father</br>
<br>Child = daughter, son, stepdaughter, stepson</br>
<br>Some children travelled only with a nanny, therefore parch=0 for them.</br>

In [ ]:
train['Parch'].value_counts()

In [ ]:
sns.catplot(x='Parch', y='Survived', kind='bar', data=train)

Possible derived features:
* 0, {1,2}, Rare

In [ ]:
train_df['parch'] = train_df['Parch'].apply(lambda x: 0 if x == 0 else (1 if (x==1 | x==2) else 2))
test_df['parch'] = test_df['Parch'].apply(lambda x: 0 if x == 0 else (1 if (x==1 | x==2) else 2))

In [ ]:
sns.catplot(x='parch', y='Survived', kind='bar', data=train_df)

## 7. Ticket

In [ ]:
train['Ticket'].tail(20)

In [ ]:
train['ticket_prefix'] = train['Ticket'].str.extract(r"(\D+)", expand=True)
train['ticket_prefix'].fillna('X')
train['ticket_prefix'].value_counts()

## 8. Fare

In [ ]:
sns.catplot(x='Survived', y='Fare', kind='box', data=train)

Median Fare paid by those who survived is higher than those who did not survive

In [ ]:
sns.distplot(train.Fare, kde=False, bins=20)

In [ ]:
sns.distplot(np.log1p(train.Fare), kde=True, bins=20)

In [ ]:
nbins = np.arange(0,700,100)
g = sns.kdeplot(train.loc[train.Survived == 0, 'Fare'], color="red", label="Not Survived", shade = True)
g = sns.kdeplot(train.loc[train.Survived == 1, 'Fare'], color="green", label="Survived", shade = True)
g.set_xlim(0,100);
g.set_xticks(nbins);

In [ ]:
nbins = np.arange(0,720,20)
hist_kws = {'bins': nbins}
hist_kws={"rwidth":0.9, 'alpha':0.3, 'bottom': 1}
g = sns.distplot(train.loc[train.Survived == 0, 'Fare'], bins=nbins, kde=False, color="red", label="Not Survived", hist_kws=hist_kws)
g = sns.distplot(train.loc[train.Survived == 1, 'Fare'], bins=nbins, kde=False, color="green", label="Survived", hist_kws=hist_kws)
g.set_xlim(0,100);
g.set_xticks(nbins);
g.set_xticklabels(nbins, rotation=90);

In [ ]:
train['Fare'] = np.log1p(train['Fare'])
test['Fare'] = np.log1p(test['Fare'])

In [ ]:
# Lets use the log of Fare as its distribution is skewed.
train_df['log_fare'] = np.log1p(train['Fare'])
test_df['log_fare'] = np.log1p(test['Fare'])

## 9. Cabin

In [ ]:
train['Cabin'].value_counts()

In [ ]:
train['Cabin'].head(10)

In [ ]:
# Cabin
# 0: Cabin absent; 1:Cabin present
train_df['has_cabin'] = train_df.Cabin.notna().astype(int)
test_df['has_cabin'] = test_df.Cabin.notna().astype(int)

In [ ]:
train_df['has_cabin'].value_counts()

In [ ]:
sns.catplot(x='has_cabin', y='Survived', kind='bar', data=train_df)

## 10. Embarked

In [ ]:
train['Embarked'].value_counts()

In [ ]:
sns.catplot(x='Embarked', y='Survived', kind = 'bar', data=train)

## 11. Age

In [ ]:
nbins = np.arange(0,105,5)
g = sns.kdeplot(train.loc[train.Survived == 0, 'Age'], color="red", label="Not Survived", shade = True)
g = sns.kdeplot(train.loc[train.Survived == 1, 'Age'], color="green", label="Survived", shade = True)
g.set_xlim(0,100);
g.set_xticks(nbins);

In [ ]:
nbins = np.arange(0,105,5)
hist_kws = {'bins': nbins}
hist_kws={"rwidth":0.9, 'alpha':0.3, 'bottom': 1}
g = sns.distplot(train.loc[train.Survived == 0, 'Age'], bins=nbins, kde=False, color="red", label="Not Survived", hist_kws=hist_kws)
g = sns.distplot(train.loc[train.Survived == 1, 'Age'], bins=nbins, kde=False, color="green", label="Survived", hist_kws=hist_kws)
g.set_xlim(0,100);
g.set_xticks(nbins);

In [ ]:
sns.catplot(x="Survived", y="Age", kind="box", data=train);

In [ ]:
# Bucketize the Age: 0-5; 5-35; 35-50; 50+
def age_bucket(age):
    if 0 <= age < 10:
        return 0
    elif 10 <= age < 35:
        return 1
    elif 35 <= age < 50:
        return 2
    else:
        return 3

# 12. Family Size - (SibSP + Parch)

In [ ]:
train_df['Family_Size'] = train['SibSp']+ train['Parch'] 

test_df['Family_Size'] = test['SibSp']+ test['Parch'] 

In [ ]:
train_df['Family_Size'].value_counts()

In [ ]:
sns.catplot(x='Family_Size', y='Survived',kind='bar' ,data=train_df)

Family size features
* 0, {1,2,3}, {4,5,6,7}

In [ ]:
train_df['familysize'] = train_df.Family_Size.apply(lambda x: 0 if x==0 else (1 if x in [1,2,3] else 2) )
test_df['familysize'] = train_df.Family_Size.apply(lambda x: 0 if x==0 else (1 if x in [1,2,3] else 2) )

In [ ]:
sns.catplot(x='familysize', y='Survived',kind='bar' ,data=train_df)

# 13. Is Alone
To represent if the person is travelling alone

In [ ]:
train_df['is_alone'] = train_df['familysize'].eq(0).astype(int)
test_df['is_alone'] = test_df['familysize'].eq(0).astype(int)

In [ ]:
train_df['is_alone'].value_counts()

In [ ]:
sns.catplot(x="is_alone", y="Survived", kind="bar", data=train_df);

# Missing Value Treatment

As seen earlier Age, Embarked & Cabin have missing values in training data whereas Age, Cabin & Fare have missing values in test data


# Pipeline

In [ ]:
train_df.info()

In [ ]:
features_to_use = ['Survived', 'Pclass', 'Sex', 'Age','SibSp','sibsp2', 'sibsp1','Parch','parch','Fare','log_fare', 'Embarked', 'Title','has_cabin','Family_Size','familysize','is_alone']
categoricals = ['sibsp1','sibsp2','parch','Embarked','Title','familysize']

In [ ]:
train_final = train_df[features_to_use]
test_final = test_df[features_to_use[1:]]

In [ ]:
train_final[categoricals] = train_final[categoricals].astype('object')
test_final[categoricals] = test_final[categoricals].astype('object')

In [ ]:
num_attribs = train_final.drop(columns='Survived').select_dtypes(include=['float64', 'int64']).columns.to_list()

In [ ]:
cat_attribs = train_final.select_dtypes(include=['object']).columns.to_list()

In [ ]:
train_final.dtypes

In [ ]:
print(num_attribs )
print(cat_attribs)

In [ ]:
train_final = train_final.sample(frac=1)
y = train_final.pop('Survived')

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown="ignore")),
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

In [ ]:
test_final.head()

In [ ]:
train_final.head()

In [ ]:
X = full_pipeline.fit_transform(train_final)
X_test = full_pipeline.transform(test_final)

In [ ]:
X_test.shape

# Let's begin with the Classification

Reusable functions

In [ ]:

# Evaluation metrics
def classification_metrics(y_true,y_pred):
    return {'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred),
            #'AUC': auc(y_true, y_pred),
            'Log Loss': log_loss(y_true, y_pred)
           }

# Cross Validation

def cross_validation(clf, X, y, scoring='accuracy', cv=10):
    scores = cross_val_score(clf, X, y, scoring=scoring, cv=cv)
    return {
        'Scores': scores,
        'Mean': scores.mean(),
        'Standard Deviation': scores.std()
    }
    

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.25, random_state=42)

## Logistic Regression Model

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)

print("==========Training Perfromace==============")
y_train_pred = lr_clf.predict(X_train)
metrics = classification_metrics(y_train, y_train_pred)
print(metrics)

print('\n')

print("==========Validation Perfromace==============")
y_val_pred = lr_clf.predict(X_val)
metrics = classification_metrics(y_val, y_val_pred)
print(metrics) 

print('\n')

# Cross Validation
scores = cross_validation(lr_clf, X_train, y_train, scoring='accuracy', cv=5)
print(scores)

In [ ]:
# Feature Selection 
model = SelectFromModel(lr_clf, prefit=True)
X_train_new = model.transform(X_train)
X_val_new = model.transform(X_val)
X_test_new = model.transform(X_test)
X_test_new.shape


In [ ]:
# Hyperparameter optimization

param_grid =[
    {'penalty': ['l2'],
     'C': [0.01,0.1,1]
     }
]

clf = LogisticRegression()

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', return_train_score=True)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
# Precision Recall Curve

y_scores = cross_val_predict(lr_clf, X_train, y_train, cv=5,method="predict_proba")
precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores[:,1])

# Plot Curve
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "r--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16) # Not shown in the book
    plt.xlabel("Threshold", fontsize=16)        # Not shown
    plt.grid(True)                              # Not shown
    #plt.axis([-50000, 50000, 0, 1])  
    
plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)

## SGD Classifier

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

print("==========Training Performance==============")
y_train_pred = sgd_clf.predict(X_train)
eval = classification_metrics(y_train_pred, y_train)
print(eval)
print('\n')
print("==========Validation Performance==============")
y_val_pred = sgd_clf.predict(X_val)
eval = classification_metrics(y_val_pred,y_val)
print(eval)
print('\n')
print("==========Cross Validation Results ================================")
scores = cross_val_score(sgd_clf, X_train, y_train,scoring='accuracy', cv=5)
print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())


In [ ]:
# Precision Recall Curve

y_scores = cross_val_predict(sgd_clf, X_train, y_train, cv=3,method="decision_function")
precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores)

# Plot Curve
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16) # Not shown in the book
    plt.xlabel("Threshold", fontsize=16)        # Not shown
    plt.grid(True)                              # Not shown
    #plt.axis([-50000, 50000, 0, 1])  
    
plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)

In [ ]:
# Precision vs Recall
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)
plt.plot([0.4368, 0.4368], [0., 0.9], "r:")
plt.plot([0.0, 0.4368], [0.9, 0.9], "r:")
plt.plot([0.4368], [0.9], "ro")
#save_fig("precision_vs_recall_plot")
plt.show()

## Decision Tree Classifier

In [ ]:
dt_clf = DecisionTreeClassifier(max_depth=3, max_features='sqrt', min_samples_leaf=25, min_samples_split=35)
dt_clf.fit(X, y)

print("==========Training Performance==============")
y_train_pred = dt_clf.predict(X_train)
eval = classification_metrics(y_train_pred, y_train)
print(eval)
print('\n')
print("==========Validation Performance==============")
y_val_pred = dt_clf.predict(X_val)
eval = classification_metrics(y_val_pred,y_val)
print(eval)
print('\n')
print("==========Cross Validation Results ================================")
scores = cross_val_score(dt_clf, X_train, y_train,scoring='accuracy', cv=5)
print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())

Hyperparameter tuning for Decision Tree

In [ ]:
param_grid = [
    {'max_depth':[3], 'max_features': ['sqrt'], 'min_samples_split':[30,35,40,45], 
     'min_samples_leaf':[25]}
]

clf = DecisionTreeClassifier()

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', return_train_score=True)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

### Random Forest Model

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=3, n_jobs=-1, random_state=42)
rf_clf.fit(X_train, y_train)

print("==========Training Perfromace==============")
y_train_pred = rf_clf.predict(X_train)
metrics = classification_metrics(y_train, y_train_pred)
print(metrics)

print('\n')

print("==========Validation Perfromace==============")
y_val_pred = rf_clf.predict(X_val)
metrics = classification_metrics(y_val, y_val_pred)
print(metrics) 

print('\n')

# Cross Validation
scores = cross_val_score(rf_clf, X_train, y_train, scoring='accuracy', cv=5)
print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())


In [ ]:
scores = cross_val_score(rf_clf, X_train, y_train,scoring='accuracy', cv=5)
print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())

## Gradient Boosting Classifier

In [ ]:
gb_clf = GradientBoostingClassifier(random_state=4)
gb_clf.fit(X_train, y_train)

print("==========Training Perfromace==============")
y_train_pred = gb_clf.predict(X_train)
metrics = classification_metrics(y_train, y_train_pred)
print(metrics)

print('\n')

print("==========Validation Perfromace==============")
y_val_pred = gb_clf.predict(X_val)
metrics = classification_metrics(y_val, y_val_pred)
print(metrics) 

print('\n')

# Cross Validation
scores = cross_val_score(gb_clf, X_train, y_train, scoring='accuracy', cv=5)
print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())


Lets try hyperparameter tuning for GBM

In [ ]:
param_grid1  = {'n_estimators':range(10,81,10)}
param_grid2 = {'max_depth':range(2,16,2), 'min_samples_split':range(30,150,30)}

clf = GradientBoostingClassifier(n_estimators = 70,
                                 learning_rate=0.1, 
                                 #min_samples_split=50,
                                 min_samples_leaf=50,
                                 #max_depth=3,
                                 max_features='sqrt',
                                 subsample=0.8,
                                 random_state=10)


grid_search = GridSearchCV(clf, param_grid2, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
grid_search.cv_results_

In [ ]:
len(X_train)

In [ ]:
X.shape

# Submission

In [ ]:
y_test = xgb_clf.predict(X_test_new)
test['Survived'] = y_test

In [ ]:
submission = test[['PassengerId', 'Survived']]
submission.to_csv('submission7.csv',mode = 'w', index=False)